Some examples of how to convert the main Diva input and output files to [geoJSON](http://geojson.org/) format.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import geojson
import pydiva2d
import matplotlib._cntr as cntr

# Input files

In [2]:
datafile = '../data/MLD1.dat'
contourfile = '../data/coast.cont'
meshfile = '../data/Mesh/BlackSea/mesh.dat'
meshtopofile = '../data/Mesh/BlackSea/meshtopo.dat'
resultfile = "../data/MLD_analysed.nc"
outputdir = "../data/geoJSON/"

We load the data and results using the `pydiva2d` module.

## Contours

In [3]:
Contour = pydiva2d.Diva2DContours().read_from(contourfile)
print(Contour.get_contours_number)

28


## Data

In [4]:
Data = pydiva2d.Diva2DData()
Data.read_from(datafile)
Data.count_data

197

## Mesh

In [5]:
Mesh = pydiva2d.Diva2DMesh()
Mesh.read_from(meshfile, meshtopofile)
Mesh.describe()

Number of nodes: 6447
Number of interfaces: 14059
Number of elements: 7592


## Field

In [6]:
Field = pydiva2d.Diva2DResults().read_from(resultfile)
Field.x.shape

(151,)

# Conversion to geoJSON

## Data

We just have to loop on the data points and add the content in the correct format.    
The field values and the weights are written as properties of the points, so that they can be represented in plots.

In [7]:
geojsondata = {
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "Point",
            "coordinates": [lon, lat],
            },
        "properties" : {"field": field, "weight": weight},
     } for lon, lat, field, weight in zip(Data.x, Data.y, Data.field, Data.weight)]
}

Then we can write the new file:

In [8]:
with open(os.path.join(outputdir, "blacksea-data.js"), 'w') as f:
    f.write("var data = ")
    out = json.dumps(geojsondata, indent=2, separators=(',', ': '))
    f.write(out)

## Contours
For the contours the best choice is a `MultiPolygon` objet.

In [9]:
geojsonContour = {
    "type": "MultiPolygon", 
    "coordinates": [[[[x, y] for x, y in zip(Contour.x[cont], Contour.y[cont])]] for cont in range(0, Contour.get_contours_number)]
}

The coutours are then written to a new file:

In [10]:
with open(os.path.join(outputdir, 'blacksea-contours.js'), 'w') as f:
    f.write("var contours = ")
    out = json.dumps(geojsonContour, indent=2, separators=(',', ': '))
    f.write(out)

## Finite-element mesh

For the mesh we also use the `MultiPolygon` object from the module geojson, but it's probably easy to do without it (i.e.: do it later!).

In [11]:
NN = Mesh.nelements
verts = []
for j in range(0, NN):
    verts.append([[(Mesh.xnode[Mesh.i1[j]], Mesh.ynode[Mesh.i1[j]]),
                  (Mesh.xnode[Mesh.i2[j]], Mesh.ynode[Mesh.i2[j]]),
                  (Mesh.xnode[Mesh.i3[j]], Mesh.ynode[Mesh.i3[j]]),
                  (Mesh.xnode[Mesh.i1[j]], Mesh.ynode[Mesh.i1[j]])]]
                )

In [12]:
geojsonMesh = geojson.MultiPolygon(verts)

We also write it to a new file:

In [13]:
with open(os.path.join(outputdir, 'blacksea-mesh.js'), 'w') as f:
    f.write("var mesh = ")
    out = json.dumps(geojsonMesh, indent=2, separators=(',', ': '))
    f.write(out)

## Field

We use the contour function of `matplotlib.pyplot` to get the iso-lines.    
Here we specify the level at which we need the contours, but we could also specify a number of contours or keep the default values of `plt.contour`.

In [14]:
llon, llat = np.meshgrid(Field.x, Field.y)
contoursField = cntr.Cntr(llon, llat, Field.analysis)
levels = np.arange(-60., -10., 5.)
geojsonfield = { 
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "MultiPolygon",
            "coordinates": [[ [ [lon, lat] for lon, lat in seg] for seg in contoursField.trace(level)[:len(contoursField.trace(level)) // 2]]],
            },
        "properties" : {"field": str(level)},
     } for level in levels] 
}

and we write it to a new file:

In [15]:
with open(os.path.join(outputdir, 'blacksea-results.js'), 'w') as f:
    f.write("var results = ")
    out = json.dumps(geojsonfield, indent=2, separators=(',', ': '))
    f.write(out)

Now all the input files are ready and we can use [Leaflet GeoJSON](http://leafletjs.com/examples/geojson/) to represent the information on a map.